In [1]:
# install.packages('tree')
library(tree)

Installing package into 'C:/Users/ongwe/Documents/R/win-library/4.1'
(as 'lib' is unspecified)



package 'tree' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ongwe\AppData\Local\Temp\RtmpkFb9NO\downloaded_packages


In [8]:
X_train <- read.csv('dengue_features_train_cleaned.csv')
X_train

city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
sj,1990,18,30/04/1990,0.1226000,0.10372500,0.1984833,0.1776167,12.42,297.5729,...,32.00,73.36571,12.42,14.01286,2.628571,25.44286,6.900000,29.4,20.0,16.0
sj,1990,19,07/05/1990,0.1699000,0.14217500,0.1623571,0.1554857,22.82,298.2114,...,17.94,77.36857,22.82,15.37286,2.371429,26.71429,6.371429,31.7,22.2,8.6
sj,1990,20,14/05/1990,0.0322500,0.17296670,0.1572000,0.1708429,34.54,298.7814,...,26.10,82.05286,34.54,16.84857,2.300000,26.71429,6.485714,32.2,22.8,41.4
sj,1990,21,21/05/1990,0.1286333,0.24506670,0.2275571,0.2358857,15.36,298.9871,...,13.90,80.33714,15.36,16.67286,2.428571,27.47143,6.771429,33.3,23.3,4.0
sj,1990,22,28/05/1990,0.1962000,0.26220000,0.2512000,0.2473400,7.52,299.5186,...,12.20,80.46000,7.52,17.21000,3.014286,28.94286,9.371429,35.0,23.9,5.8
sj,1990,23,04/06/1990,0.1545500,0.17485000,0.2543143,0.1817429,9.58,299.6300,...,26.49,79.89143,9.58,17.21286,2.100000,28.11429,6.942857,34.4,23.9,39.1
sj,1990,24,11/06/1990,0.1129000,0.09280000,0.2050714,0.2102714,3.48,299.2071,...,38.60,82.00000,3.48,17.23429,2.042857,27.41429,6.771429,32.2,23.3,29.7
sj,1990,25,18/06/1990,0.0725000,0.07250000,0.1514714,0.1330286,151.12,299.5914,...,30.00,83.37571,151.12,17.97714,1.571429,28.37143,7.685714,33.9,22.8,21.1
sj,1990,26,25/06/1990,0.1024500,0.14617500,0.1255714,0.1236000,19.32,299.5786,...,37.51,82.76857,19.32,17.79000,1.885714,28.32857,7.385714,33.9,22.8,21.1


In [16]:
train_labels <- read.csv('dengue_features_train.csv')


'data.frame':	1456 obs. of  24 variables:
 $ city                                 : chr  "sj" "sj" "sj" "sj" ...
 $ year                                 : int  1990 1990 1990 1990 1990 1990 1990 1990 1990 1990 ...
 $ weekofyear                           : int  18 19 20 21 22 23 24 25 26 27 ...
 $ week_start_date                      : chr  "30/04/1990" "07/05/1990" "14/05/1990" "21/05/1990" ...
 $ ndvi_ne                              : num  0.1226 0.1699 0.0323 0.1286 0.1962 ...
 $ ndvi_nw                              : num  0.104 0.142 0.173 0.245 0.262 ...
 $ ndvi_se                              : num  0.198 0.162 0.157 0.228 0.251 ...
 $ ndvi_sw                              : num  0.178 0.155 0.171 0.236 0.247 ...
 $ precipitation_amt_mm                 : num  12.42 22.82 34.54 15.36 7.52 ...
 $ reanalysis_air_temp_k                : num  298 298 299 299 300 ...
 $ reanalysis_avg_temp_k                : num  298 298 299 299 300 ...
 $ reanalysis_dew_point_temp_k          : num  292 